In [1]:
import json
from pathlib import Path

# Set the paths
input_json = "coco/annotations/instances_train2017.json"
output_json = "coco/annotations/instances_vehicles_only.json"
required_categories = ['car', 'motorcycle', 'bus', 'truck']

# Load original COCO annotations
with open(input_json, 'r') as f:
    coco_data = json.load(f)

# Step 1: Filter categories
category_ids = {cat['id']: cat['name'] for cat in coco_data['categories']}
filtered_category_ids = [cat_id for cat_id, name in category_ids.items() if name in required_categories]

# Step 2: Filter annotations
filtered_annotations = [ann for ann in coco_data['annotations'] if ann['category_id'] in filtered_category_ids]
image_ids = list({ann['image_id'] for ann in filtered_annotations})

# Step 3: Filter images
filtered_images = [img for img in coco_data['images'] if img['id'] in image_ids]

# Step 4: Filter categories list
filtered_categories = [cat for cat in coco_data['categories'] if cat['id'] in filtered_category_ids]

# Step 5: Combine and save
filtered_coco = {
    'images': filtered_images,
    'annotations': filtered_annotations,
    'categories': filtered_categories
}

with open(output_json, 'w') as f:
    json.dump(filtered_coco, f)

print(f"✅ Written {len(filtered_annotations)} annotations across {len(filtered_images)} images to {output_json}")


✅ Written 68634 annotations across 18175 images to coco/annotations/instances_vehicles_only.json


In [3]:
import json
import os
import shutil
from pathlib import Path
from sklearn.model_selection import train_test_split
from tqdm import tqdm

# ────────────────────────────────────────────────────────────────────────────────
# CONFIGURATION: adjust paths if needed
COCO_JSON   = Path("coco/annotations/instances_vehicles_only.json")
COCO_IMGDIR = Path("coco/train2017")              
OUT_DIR     = Path("yolodata")                     # output root for YOLO data
AUTO_DIR    = Path("path/to/your/Auto")            
SPLIT_RATIO = 0.1                                  # 10% of vehicle images → validation
# ────────────────────────────────────────────────────────────────────────────────

# Make sure output folders exist
for split in ["train", "val"]:
    (OUT_DIR/ split/ "images").mkdir(parents=True, exist_ok=True)
    (OUT_DIR/ split/ "labels").mkdir(parents=True, exist_ok=True)

# Load the filtered COCO JSON
with open(COCO_JSON, "r") as f:
    data = json.load(f)

# Build maps: image_id → (file_name, width, height)
img_map = { img["id"]: (img["file_name"], img["width"], img["height"])
            for img in data["images"] }

# Group annotations by image_id
ann_by_img = {}
for ann in data["annotations"]:
    ann_by_img.setdefault(ann["image_id"], []).append(ann)

# Create a list of all filtered image_ids
all_img_ids = list(img_map.keys())

# Split into train / val
train_ids, val_ids = train_test_split(all_img_ids, test_size=SPLIT_RATIO, random_state=42)

def process_set(img_ids, split):
    """
    For each image_id in img_ids:
      - copy the JPEG from COCO_IMGDIR → OUT_DIR/{split}/images/
      - write a YOLO .txt file into OUT_DIR/{split}/labels/
    """
    for img_id in tqdm(img_ids, desc=f"Processing {split} images"):
        fn, w, h = img_map[img_id]
        src_img = COCO_IMGDIR / fn
        dst_img = OUT_DIR/ split/ "images"/ fn

        # Copy the image
        if not dst_img.exists():
            shutil.copy(src_img, dst_img)

        # Convert each annotation to YOLO format
        anns = ann_by_img.get(img_id, [])
        lines = []
        for a in anns:
            # Map category_id to a class index 0–3:
            #  Suppose we assign:
            #    0 = car
            #    1 = motorcycle
            #    2 = bus
            #    3 = truck
            cat_to_cls = {3: 0, 4: 1, 6: 2, 8: 3}
            cls_idx = cat_to_cls[a["category_id"]]

            x, y, bw, bh = a["bbox"]
            xc = (x + bw/2) / w
            yc = (y + bh/2) / h
            nw = bw / w
            nh = bh / h

            lines.append(f"{cls_idx} {xc:.6f} {yc:.6f} {nw:.6f} {nh:.6f}")

        # Write the .txt labels
        lbl_name = fn.replace(".jpg", ".txt")
        with open(OUT_DIR/ split/ "labels"/ lbl_name, "w") as lf:
            lf.write("\n".join(lines))

# Process train split
process_set(train_ids, "train")

# Process val split
process_set(val_ids, "val")

print(f"✅ Converted {len(train_ids)} train images + {len(val_ids)} val images to YOLO format.")


Processing val images: 100%|██████████| 1818/1818 [00:07<00:00, 228.36it/s]

✅ Converted 16357 train images + 1818 val images to YOLO format.


In [4]:
import shutil
from pathlib import Path

# ── CONFIG: point AUTO_DIR ───────────────────────
AUTO_DIR = Path("D:\Machine Learning\Major Project\Auto")  
YOLO_ROOT = Path("yolodata")          # the root where train/ and val/ exist
# ──────────────────────────────────────────────────────────────────────────────

img_dst = YOLO_ROOT / "train" / "images"
lbl_dst = YOLO_ROOT / "train" / "labels"

# Loop over each .txt in AUTO_DIR, rewrite its class to “4”, then copy .jpg/.txt
for txt_path in AUTO_DIR.glob("*.txt"):
    # Read existing lines and replace the class index (first token) with “4”
    new_lines = []
    for line in txt_path.read_text().splitlines():
        parts = line.split()
        parts[0] = "4"  # set class index to 4 (next available after {0,1,2,3})
        new_lines.append(" ".join(parts))
    # Write the modified label into the train/labels folder
    (lbl_dst / txt_path.name).write_text("\n".join(new_lines))

    # Copy the corresponding image (.jpg) into train/images
    img_name = txt_path.with_suffix(".jpg").name
    shutil.copy(AUTO_DIR / img_name, img_dst / img_name)

print("✅ Merged all autos into:", img_dst)


✅ Merged all autos into: yolodata\train\images


In [5]:
from pathlib import Path

YOLO_ROOT = Path("yolodata")
data_yaml = f"""\
train: { (YOLO_ROOT / 'train' / 'images').resolve() }
val:   { (YOLO_ROOT / 'val'   / 'images').resolve() }

nc: 5
names: ['car','motorcycle','bus','truck','auto']
"""

# Write to yolodata/data.yaml
(Path(YOLO_ROOT) / "data.yaml").write_text(data_yaml)
print("✅ Wrote data.yaml to", YOLO_ROOT / "data.yaml")


✅ Wrote data.yaml to yolodata\data.yaml


In [6]:
from pathlib import Path
from collections import Counter

val_labels = Path("yolodata/val/labels")
class_counts = Counter()

for txt_file in val_labels.glob("*.txt"):
    lines = txt_file.read_text().splitlines()
    for line in lines:
        cls = int(line.split()[0])
        class_counts[cls] += 1

print("Class counts in val set:", dict(class_counts))


Class counts in val set: {2: 595, 0: 4278, 1: 938, 3: 996}


In [8]:
import random, shutil
from pathlib import Path

# ── CONFIG ────────────────────────────────────────────────────────────────
YOLO_ROOT    = Path("yolodata")
AUTO_CLASS   = 4            # class index for “auto”
REQUESTED_MOVE = 40         # desired number of autos to move to val
# ────────────────────────────────────────────────────────────────────────────

# Paths to train/val images & labels
train_images = YOLO_ROOT / "train" / "images"
train_labels = YOLO_ROOT / "train" / "labels"
val_images   = YOLO_ROOT / "val"   / "images"
val_labels   = YOLO_ROOT / "val"   / "labels"

# 1) Collect all train-label files whose first token == AUTO_CLASS
auto_txts = []
for txt in train_labels.glob("*.txt"):
    first_line = txt.read_text().splitlines()
    if not first_line: 
        continue
    first_token = first_line[0].split()[0]
    if int(first_token) == AUTO_CLASS:
        auto_txts.append(txt)

total_autos = len(auto_txts)
print(f"Found {total_autos} auto labels in train set.")

move_count = min(REQUESTED_MOVE, total_autos)
if move_count < REQUESTED_MOVE:
    print(f"Only {total_autos} autos available; moving all of them.")

random.seed(42)
to_move = random.sample(auto_txts, move_count)

for txt_path in to_move:
    img_name = txt_path.with_suffix(".jpg").name
    # Move label
    shutil.move(txt_path, val_labels / txt_path.name)
    # Move image
    shutil.move(train_images / img_name, val_images / img_name)

print(f"✅ Moved {len(to_move)} auto images + labels to val split.")


Found 0 auto labels in train set.
Only 0 autos available; moving all of them.
✅ Moved 0 auto images + labels to val split.


In [9]:
from pathlib import Path

AUTO_DIR = Path("D:\Machine Learning\Major Project\Auto") 
print("Files in AUTO_DIR (first 10):")
for i, f in enumerate(sorted(AUTO_DIR.iterdir())):
    if i >= 10: break
    print(" ", f.name)
  
# Also peek inside one .txt to see its class index
some_txt = next(AUTO_DIR.glob("*.txt"), None)
print("\nSample .txt content:", some_txt.read_text().splitlines()[0] if some_txt else "No .txt found")


Files in AUTO_DIR (first 10):
  Datacluster Auto (1).jpg
  Datacluster Auto (1).xml
  Datacluster Auto (10).jpg
  Datacluster Auto (10).xml
  Datacluster Auto (100).jpg
  Datacluster Auto (100).xml
  Datacluster Auto (101).jpg
  Datacluster Auto (101).xml
  Datacluster Auto (102).jpg
  Datacluster Auto (102).xml

Sample .txt content: No .txt found


In [10]:
import xml.etree.ElementTree as ET
from pathlib import Path
import shutil

AUTO_DIR = Path("D:\Machine Learning\Major Project\Auto")
YOLO_TRAIN_IMAGES = Path("yolodata/train/images")
YOLO_TRAIN_LABELS = Path("yolodata/train/labels")
# Ensure these exist
YOLO_TRAIN_IMAGES.mkdir(parents=True, exist_ok=True)
YOLO_TRAIN_LABELS.mkdir(parents=True, exist_ok=True)

AUTO_CLASS_IDX = 4

xml_files = list(AUTO_DIR.glob("*.xml"))
print(f"Found {len(xml_files)} XML files to convert.")

for xml_path in xml_files:
    tree = ET.parse(xml_path)
    root = tree.getroot()

    # Read image filename & size from XML
    img_filename = root.findtext("filename")
    size = root.find("size")
    width = float(size.findtext("width"))
    height = float(size.findtext("height"))

    # Collect YOLO lines
    yolo_lines = []
    for obj in root.findall("object"):
        cls_name = obj.findtext("name")
        if cls_name.lower() != "auto":
            continue

        bnd = obj.find("bndbox")
        xmin = float(bnd.findtext("xmin"))
        ymin = float(bnd.findtext("ymin"))
        xmax = float(bnd.findtext("xmax"))
        ymax = float(bnd.findtext("ymax"))

        # Convert to YOLO format (normalized center, width, height)
        x_center = ((xmin + xmax) / 2) / width
        y_center = ((ymin + ymax) / 2) / height
        w_norm = (xmax - xmin) / width
        h_norm = (ymax - ymin) / height

        yolo_lines.append(f"{AUTO_CLASS_IDX} {x_center:.6f} {y_center:.6f} {w_norm:.6f} {h_norm:.6f}")

    if not yolo_lines:
        continue

    # Write .txt in train/labels
    txt_name = Path(img_filename).with_suffix(".txt")
    (YOLO_TRAIN_LABELS / txt_name).write_text("\n".join(yolo_lines))

    # Copy the .jpg into train/images
    src_img = AUTO_DIR / img_filename
    dst_img = YOLO_TRAIN_IMAGES / img_filename
    if src_img.exists():
        shutil.copy(src_img, dst_img)
    else:
        print(f"⚠️ Image not found for {xml_path.name}: expected {img_filename}")

print("✅ Conversion complete: XML → YOLO TXT and images copied to yolodata/train.")


Found 152 XML files to convert.
✅ Conversion complete: XML → YOLO TXT and images copied to yolodata/train.


In [11]:
from pathlib import Path
from collections import Counter

lbls = list((Path("yolodata/train/labels")).glob("*.txt"))
print(f"Total auto TXT files created: {len(lbls)}")

counts = Counter()
for txt in lbls:
    lines = txt.read_text().splitlines()
    for ln in lines:
        cls = int(ln.split()[0])
        counts[cls] += 1

print("Label-class counts (should show only {4: some_number}):", dict(counts))


Total auto TXT files created: 16509
Label-class counts (should show only {4: some_number}): {0: 39589, 3: 8977, 1: 7787, 2: 5474, 4: 236}


In [12]:
import random, shutil
from pathlib import Path

YOLO_ROOT     = Path("yolodata")
AUTO_CLASS    = 4
REQUESTED_NUM = 40

train_images = YOLO_ROOT / "train" / "images"
train_labels = YOLO_ROOT / "train" / "labels"
val_images   = YOLO_ROOT / "val"   / "images"
val_labels   = YOLO_ROOT / "val"   / "labels"

# Find auto label files
auto_txts = []
for txt in train_labels.glob("*.txt"):
    first = txt.read_text().splitlines()
    if first and int(first[0].split()[0]) == AUTO_CLASS:
        auto_txts.append(txt)

move_n = min(REQUESTED_NUM, len(auto_txts))
print(f"Moving {move_n}/{len(auto_txts)} autos to val")

to_move = random.sample(auto_txts, move_n)
for txt in to_move:
    img = train_images / txt.with_suffix(".jpg").name
    shutil.move(txt, val_labels / txt.name)
    shutil.move(img, val_images / img.name)

print("✅ Moved selected autos to val.")


Moving 40/152 autos to val
✅ Moved selected autos to val.


In [13]:
from pathlib import Path
from collections import Counter

val_lbls = list((Path("yolodata/val/labels")).glob("*.txt"))
counts = Counter()
for txt in val_lbls:
    for ln in txt.read_text().splitlines():
        cls = int(ln.split()[0])
        counts[cls] += 1
print("Val‐set class counts:", dict(counts))


Val‐set class counts: {2: 595, 0: 4278, 1: 938, 3: 996, 4: 60}


In [14]:
from pathlib import Path

YOLO_ROOT = Path("yolodata")
content = f"""\
train: { (YOLO_ROOT / 'train' / 'images').resolve() }
val:   { (YOLO_ROOT / 'val'   / 'images').resolve() }

nc: 5
names: ['car','motorcycle','bus','truck','auto']
"""

(YOLO_ROOT / "data.yaml").write_text(content)
print("✅ data.yaml updated:", YOLO_ROOT / "data.yaml")


✅ data.yaml updated: yolodata\data.yaml


In [15]:
import random, shutil
from pathlib import Path

# ── CONFIG ──────────────────────────────────────────────────────────────────────
YOLO_ROOT   = Path("yolodata")
AUTO_CLASS  = 4            
VAL_RATIO   = 0.15           # ~15% for validation
# ────────────────────────────────────────────────────────────────────────────────

auto_train_imgs = YOLO_ROOT / "train" / "images"
auto_train_lbls = YOLO_ROOT / "train" / "labels"

# Create new folders
AUTO_ROOT = YOLO_ROOT / "auto"
for sub in ["train/images", "train/labels", "val/images", "val/labels"]:
    (AUTO_ROOT / Path(sub)).mkdir(parents=True, exist_ok=True)

# Gather all “auto” label files from the main train split
all_auto_txts = []
for txt in auto_train_lbls.glob("*.txt"):
    first = txt.read_text().splitlines()
    if first and int(first[0].split()[0]) == AUTO_CLASS:
        all_auto_txts.append(txt)

# Shuffle and split
random.seed(42)
random.shuffle(all_auto_txts)
split_idx = int(len(all_auto_txts) * (1 - VAL_RATIO))
train_auto = all_auto_txts[:split_idx]
val_auto   = all_auto_txts[split_idx:]

# Function to copy .jpg/.txt into target
def copy_auto(txt_list, subset):
    for txt in txt_list:
        img_name = txt.with_suffix(".jpg").name
        # Read original TXT content (already “4 x y w h”)
        content = txt.read_text().splitlines()
        # Rewrite class index to 0 for auto-only
        new_lines = []
        for line in content:
            parts = line.split()
            parts[0] = "0"
            new_lines.append(" ".join(parts))
        # Write to auto/{subset}/labels
        dest_lbl = AUTO_ROOT / subset / "labels" / txt.name
        dest_lbl.write_text("\n".join(new_lines))
        # Copy image to auto/{subset}/images
        shutil.copy(auto_train_imgs / img_name, AUTO_ROOT / subset / "images" / img_name)

# Copy training autos
copy_auto(train_auto, "train")
# Copy validation autos
copy_auto(val_auto, "val")

print(f"✅ Auto-only: {len(train_auto)} in train, {len(val_auto)} in val")


✅ Auto-only: 95 in train, 17 in val


In [16]:
from pathlib import Path

AUTO_ROOT = Path("yolodata/auto")

train_imgs = list((AUTO_ROOT/"train"/"images").glob("*.jpg"))
train_lbls = list((AUTO_ROOT/"train"/"labels").glob("*.txt"))
val_imgs   = list((AUTO_ROOT/"val"/"images").glob("*.jpg"))
val_lbls   = list((AUTO_ROOT/"val"/"labels").glob("*.txt"))

print("Auto‐only train images:", len(train_imgs))
print("Auto‐only train labels:", len(train_lbls))
print("Auto‐only  val images:", len(val_imgs))
print("Auto‐only  val labels:", len(val_lbls))


Auto‐only train images: 95
Auto‐only train labels: 95
Auto‐only  val images: 17
Auto‐only  val labels: 17


In [17]:
from pathlib import Path

AUTO_ROOT = Path("yolodata/auto")
content = f"""\
train: { (AUTO_ROOT / 'train' / 'images').resolve() }
val:   { (AUTO_ROOT / 'val'   / 'images').resolve() }

nc: 1
names: ['auto']
"""

(AUTO_ROOT / "auto_only.yaml").write_text(content)
print("✅ Wrote auto_only.yaml to", AUTO_ROOT / "auto_only.yaml")


✅ Wrote auto_only.yaml to yolodata\auto\auto_only.yaml


In [18]:
# ────────────────────────────────────────────────────────────────────────────────
# Step 5: Oversample “auto” images in yolodata/train so they’re not underrepresented
# ────────────────────────────────────────────────────────────────────────────────

import shutil
from pathlib import Path

# Configuration: where your combined dataset lives
YOLO_ROOT = Path("yolodata")
TRAIN_IMGS = YOLO_ROOT / "train" / "images"
TRAIN_LBLS = YOLO_ROOT / "train" / "labels"
AUTO_CLASS = 4  # class index for “auto”
DUP_FACTOR = 3  # how many additional copies of each auto to make
# ────────────────────────────────────────────────────────────────────────────────

# Find all “auto” label files in train/labels
auto_txts = []
for txt in TRAIN_LBLS.glob("*.txt"):
    first = txt.read_text().splitlines()
    if first and int(first[0].split()[0]) == AUTO_CLASS:
        auto_txts.append(txt)

print(f"Found {len(auto_txts)} original auto labels in train.")

# For each auto, create DUP_FACTOR copies with new names
for txt_path in auto_txts:
    base = txt_path.stem  
    img_name = f"{base}.jpg"
    src_img = TRAIN_IMGS / img_name

    for i in range(DUP_FACTOR):
        new_base = f"{base}_dup{i}"
        # Copy image
        dest_img = TRAIN_IMGS / f"{new_base}.jpg"
        shutil.copy(src_img, dest_img)
        # Copy label (same contents)
        dest_lbl = TRAIN_LBLS / f"{new_base}.txt"
        shutil.copy(txt_path, dest_lbl)

print(f"✅ Duplicated each auto {DUP_FACTOR} times.")


Found 112 original auto labels in train.
✅ Duplicated each auto 3 times.


In [ ]:
# Step: Augment “auto” images by flipping to balance class counts

from pathlib import Path
from PIL import Image
import shutil
import uuid

# ────────────────────────────────────────────────────────────────────────────────
# CONFIGURATION
YOLO_ROOT    = Path("yolodata")
TRAIN_IMAGES = YOLO_ROOT / "train" / "images"
TRAIN_LABELS = YOLO_ROOT / "train" / "labels"
AUTO_CLASS   = 4            # class index for “auto”
AUG_DIR      = TRAIN_IMAGES # we’ll write augmented images/labels back into train folders
# ────────────────────────────────────────────────────────────────────────────────

def augment_autos():
    # 1) Gather all original “auto” label files
    auto_label_files = []
    for txt in TRAIN_LABELS.glob("*.txt"):
        lines = txt.read_text().splitlines()
        if not lines:
            continue
        cls = int(lines[0].split()[0])
        if cls == AUTO_CLASS:
            auto_label_files.append(txt)

    print(f"Found {len(auto_label_files)} auto .txt files to augment.")

    for txt_path in auto_label_files:
        # Read the corresponding image
        base_name = txt_path.stem
        img_path = TRAIN_IMAGES / f"{base_name}.jpg"
        if not img_path.exists():
            continue

        # Load image and label lines
        img = Image.open(img_path)
        w, h = img.size
        label_lines = txt_path.read_text().splitlines()

        # Perform 3 augmentations: horizontal flip, vertical flip, both
        ops = [
            ("hflip",   lambda im: im.transpose(Image.FLIP_LEFT_RIGHT)),
            ("vflip",   lambda im: im.transpose(Image.FLIP_TOP_BOTTOM)),
            ("hvflip",  lambda im: im.transpose(Image.FLIP_LEFT_RIGHT).transpose(Image.FLIP_TOP_BOTTOM)),
        ]

        for suffix, op in ops:
            # 2) Create augmented image
            aug_img = op(img)
            new_basename = f"{base_name}_{suffix}_{uuid.uuid4().hex[:8]}"
            new_img_path = AUG_DIR / f"{new_basename}.jpg"
            aug_img.save(new_img_path)

            # 3) Adjust and write label file
            new_txt_path = TRAIN_LABELS / f"{new_basename}.txt"
            new_lines = []
            for line in label_lines:
                parts = line.split()
                cls_idx = parts[0]  # should be “4”
                xc, yc, bw, bh = map(float, parts[1:5])

                if suffix == "hflip":
                    xc = 1.0 - xc
                elif suffix == "vflip":
                    yc = 1.0 - yc
                elif suffix == "hvflip":
                    xc = 1.0 - xc
                    yc = 1.0 - yc

                new_lines.append(f"{cls_idx} {xc:.6f} {yc:.6f} {bw:.6f} {bh:.6f}")

            new_txt_path.write_text("\n".join(new_lines))

    print("✅ Augmentation complete: horizontal, vertical, and both flips applied.")


# Run the augmentation
augment_autos()


Found 1125 auto .txt files to augment.


In [3]:
from pathlib import Path
from PIL import Image
import shutil
import uuid

# CONFIGURATION: paths and settings
YOLO_ROOT    = Path("yolodata")
TRAIN_IMAGES = YOLO_ROOT / "train" / "images"
TRAIN_LABELS = YOLO_ROOT / "train" / "labels"
AUTO_CLASS   = 4  

# Suffixes used by the previous augmentation that we want to skip
SKIP_SUFFIXES = ["_hflip_", "_vflip_", "_hvflip_"]


def needs_augmentation(txt_path):
    """
    Return True if:
    - The label’s first token is AUTO_CLASS (4), and
    - The stem does not contain any of the SKIP_SUFFIXES.
    """
    stem = txt_path.stem
    for suffix in SKIP_SUFFIXES:
        if suffix in stem:
            return False

    # Check the class index is AUTO_CLASS
    lines = txt_path.read_text().splitlines()
    if not lines:
        return False
    cls_idx = int(lines[0].split()[0])
    return cls_idx == AUTO_CLASS

def augment_autos_once():
    # 1) Gather only those auto labels that still need augmentation
    to_augment = [txt for txt in TRAIN_LABELS.glob("*.txt") if needs_augmentation(txt)]
    print(f"Found {len(to_augment)} auto files needing augmentation.")

    for txt_path in to_augment:
        # Load corresponding image
        base_name = txt_path.stem
        img_path = TRAIN_IMAGES / f"{base_name}.jpg"
        if not img_path.exists():
            continue  # skip if image missing

        img = Image.open(img_path)
        w, h = img.size

        # Read the label lines (all start with "4 x y w h")
        label_lines = txt_path.read_text().splitlines()

        # Define flip operations & suffixes
        ops = [
            ("_hflip_",  lambda im: im.transpose(Image.FLIP_LEFT_RIGHT)),
            ("_vflip_",  lambda im: im.transpose(Image.FLIP_TOP_BOTTOM)),
            ("_hvflip_", lambda im: im.transpose(Image.FLIP_LEFT_RIGHT).transpose(Image.FLIP_TOP_BOTTOM)),
        ]

        for suffix, op in ops:
            aug_img = op(img)
            new_basename = f"{base_name}{suffix}{uuid.uuid4().hex[:8]}"
            new_img_path = TRAIN_IMAGES / f"{new_basename}.jpg"
            aug_img.save(new_img_path)

            new_txt_path = TRAIN_LABELS / f"{new_basename}.txt"
            new_lines = []
            for line in label_lines:
                parts = line.split()
                # keep class 4
                xc, yc, bw, bh = map(float, parts[1:5])

                # adjust centers for flip
                if suffix == "_hflip_":
                    xc = 1.0 - xc
                elif suffix == "_vflip_":
                    yc = 1.0 - yc
                elif suffix == "_hvflip_":
                    xc = 1.0 - xc
                    yc = 1.0 - yc

                new_lines.append(f"4 {xc:.6f} {yc:.6f} {bw:.6f} {bh:.6f}")

            new_txt_path.write_text("\n".join(new_lines))

    print("✅ Finished augmenting those auto files.")


# Run the augmentation pass
augment_autos_once()


Found 448 auto files needing augmentation.


OSError: image file is truncated (43 bytes not processed)

In [4]:
from pathlib import Path
from collections import Counter

counts = Counter()
for txt in (TRAIN_LABELS).glob("*.txt"):
    cls = int(txt.read_text().split()[0])
    counts[cls] += 1

print("Post‐re‐augmentation train class counts:", dict(counts))


Post‐re‐augmentation train class counts: {0: 9975, 1: 2134, 3: 2324, 2: 1924, 4: 2111}
